# Yu Darvish 2025 Season Analysis

**Statcast Data Analysis with Unique Perspectives**

- Monthly Performance Changes
- Toughest Batters Analysis
- Fatigue Patterns (Velocity Drop by Inning)
- Count-Based Pitch Selection
- Whiff Rate by Pitch Type
- Batted Ball Damage Analysis
- L/R Splits

In [ ]:
!pip install pybaseball duckdb -q

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from pybaseball import statcast_pitcher, playerid_lookup
import duckdb

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# ====== Settings ======
PITCHER_ID = 506433  # Yu Darvish MLBAM ID
SEASON_YEAR = 2025
GAME_TYPE = "R"  # "R"=Regular, "P"=Postseason, None=All
# ======================

## 1. Data Acquisition

In [ ]:
# Fetch Darvish's 2025 pitching data
START_DATE = f"{SEASON_YEAR}-03-01"
END_DATE = f"{SEASON_YEAR}-12-31"

print(f"Fetching data for Yu Darvish ({PITCHER_ID})...")
print(f"Period: {START_DATE} to {END_DATE}")

df_raw = statcast_pitcher(START_DATE, END_DATE, PITCHER_ID)
print(f"Total records (raw): {len(df_raw):,}")

# Filter by game type
con = duckdb.connect()

if GAME_TYPE:
    df = con.execute(f"""
        SELECT *
        FROM df_raw
        WHERE game_type = '{GAME_TYPE}'
    """).df()
    print(f"Filtered records (game_type='{GAME_TYPE}'): {len(df):,}")
else:
    df = df_raw.copy()
    print(f"Using all game types: {len(df):,}")

# Show unique pitch types
print(f"\nPitch Types: {sorted(df['pitch_type'].dropna().unique())}")
print(f"Game Dates: {df['game_date'].nunique()} games")

## 2. Season Overview

In [ ]:
# Basic stats overview
overview = con.execute("""
    SELECT
        COUNT(*) as total_pitches,
        COUNT(DISTINCT game_date) as games,
        ROUND(AVG(release_speed), 1) as avg_velo,
        ROUND(MAX(release_speed), 1) as max_velo,
        ROUND(AVG(release_spin_rate), 0) as avg_spin,
        ROUND(AVG(pfx_x) * 12, 1) as avg_h_break_in,
        ROUND(AVG(pfx_z) * 12, 1) as avg_v_break_in
    FROM df
""").df()

print("=== 2025 Season Overview ===")
print(overview.T.to_string())

In [ ]:
# Pitch type distribution
pitch_dist = con.execute("""
    SELECT
        pitch_type,
        COUNT(*) as count,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 1) as pct,
        ROUND(AVG(release_speed), 1) as avg_velo,
        ROUND(AVG(release_spin_rate), 0) as avg_spin
    FROM df
    WHERE pitch_type IS NOT NULL
    GROUP BY pitch_type
    ORDER BY count DESC
""").df()

print("\n=== Pitch Type Distribution ===")
print(pitch_dist.to_string(index=False))

# Pie chart
fig, ax = plt.subplots(figsize=(8, 8))
ax.pie(pitch_dist['count'], labels=pitch_dist['pitch_type'], autopct='%.1f%%',
       colors=plt.cm.Set3.colors)
ax.set_title('Yu Darvish 2025 - Pitch Type Distribution')
plt.show()

## 3. Monthly Performance Changes

In [ ]:
# Monthly velocity and spin rate trends
monthly_stats = con.execute("""
    SELECT
        strftime(game_date, '%Y-%m') as month,
        pitch_type,
        COUNT(*) as pitches,
        ROUND(AVG(release_speed), 1) as avg_velo,
        ROUND(AVG(release_spin_rate), 0) as avg_spin
    FROM df
    WHERE pitch_type IS NOT NULL
    GROUP BY month, pitch_type
    ORDER BY month, pitches DESC
""").df()

print("=== Monthly Stats by Pitch Type ===")
print(monthly_stats.to_string(index=False))

In [ ]:
# Plot: Monthly velocity trend for main pitches
main_pitches = pitch_dist.head(4)['pitch_type'].tolist()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for pitch in main_pitches:
    data = monthly_stats[monthly_stats['pitch_type'] == pitch]
    axes[0].plot(data['month'], data['avg_velo'], marker='o', label=pitch)
    axes[1].plot(data['month'], data['avg_spin'], marker='o', label=pitch)

axes[0].set_title('Monthly Velocity Trend')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Velocity (mph)')
axes[0].legend()
axes[0].tick_params(axis='x', rotation=45)

axes[1].set_title('Monthly Spin Rate Trend')
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Spin Rate (rpm)')
axes[1].legend()
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Monthly pitch mix change
monthly_mix = con.execute("""
    SELECT
        strftime(game_date, '%Y-%m') as month,
        pitch_type,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(PARTITION BY strftime(game_date, '%Y-%m')), 1) as pct
    FROM df
    WHERE pitch_type IS NOT NULL
    GROUP BY month, pitch_type
    ORDER BY month
""").df()

# Pivot for stacked bar
pivot_mix = monthly_mix.pivot(index='month', columns='pitch_type', values='pct').fillna(0)

pivot_mix.plot(kind='bar', stacked=True, figsize=(12, 6), colormap='Set3')
plt.title('Yu Darvish 2025 - Monthly Pitch Mix')
plt.xlabel('Month')
plt.ylabel('Percentage (%)')
plt.legend(title='Pitch Type', bbox_to_anchor=(1.05, 1))
plt.tight_layout()
plt.show()

## 4. Toughest Batters Analysis

In [ ]:
# Batters who hit well against Darvish
# events: single, double, triple, home_run = hits
batter_stats = con.execute("""
    WITH at_bats AS (
        SELECT
            batter,
            COUNT(DISTINCT CASE WHEN events IS NOT NULL THEN game_pk || '-' || at_bat_number END) as PA,
            SUM(CASE WHEN events IN ('single', 'double', 'triple', 'home_run') THEN 1 ELSE 0 END) as hits,
            SUM(CASE WHEN events = 'strikeout' THEN 1 ELSE 0 END) as strikeouts,
            SUM(CASE WHEN events IN ('walk', 'hit_by_pitch') THEN 1 ELSE 0 END) as walks,
            SUM(CASE WHEN events = 'home_run' THEN 1 ELSE 0 END) as hr,
            ROUND(AVG(CASE WHEN launch_speed IS NOT NULL THEN launch_speed END), 1) as avg_exit_velo
        FROM df
        GROUP BY batter
        HAVING PA >= 3  -- At least 3 PA
    )
    SELECT
        batter,
        PA,
        hits,
        strikeouts,
        walks,
        hr,
        ROUND(1.0 * hits / NULLIF(PA - walks, 0), 3) as AVG,
        avg_exit_velo
    FROM at_bats
    ORDER BY AVG DESC
    LIMIT 15
""").df()

print("=== Toughest Batters (by AVG, min 3 PA) ===")
print(batter_stats.to_string(index=False))

In [ ]:
# Hardest hit balls against Darvish
hard_hit = con.execute("""
    SELECT
        batter,
        game_date,
        events,
        pitch_type,
        ROUND(launch_speed, 1) as exit_velo,
        ROUND(launch_angle, 1) as launch_angle,
        ROUND(release_speed, 1) as pitch_velo
    FROM df
    WHERE launch_speed IS NOT NULL
    ORDER BY launch_speed DESC
    LIMIT 10
""").df()

print("\n=== Hardest Hit Balls Against Darvish ===")
print(hard_hit.to_string(index=False))

## 5. Fatigue Analysis (Velocity Drop by Inning)

In [ ]:
# Velocity by inning
inning_velo = con.execute("""
    SELECT
        inning,
        pitch_type,
        COUNT(*) as pitches,
        ROUND(AVG(release_speed), 1) as avg_velo,
        ROUND(AVG(release_spin_rate), 0) as avg_spin
    FROM df
    WHERE pitch_type IS NOT NULL AND inning <= 9
    GROUP BY inning, pitch_type
    ORDER BY inning, pitches DESC
""").df()

# Plot for fastball (FF) or main pitch
ff_data = inning_velo[inning_velo['pitch_type'] == 'FF'] if 'FF' in inning_velo['pitch_type'].values else inning_velo[inning_velo['pitch_type'] == inning_velo.iloc[0]['pitch_type']]

fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(ff_data['inning'], ff_data['avg_velo'], color='steelblue', alpha=0.7)
ax.set_xlabel('Inning')
ax.set_ylabel('Velocity (mph)')
ax.set_title(f'Yu Darvish 2025 - Fastball Velocity by Inning')
ax.set_xticks(range(1, 10))

# Add trend line
if len(ff_data) > 1:
    z = np.polyfit(ff_data['inning'], ff_data['avg_velo'], 1)
    p = np.poly1d(z)
    ax.plot(ff_data['inning'], p(ff_data['inning']), "r--", alpha=0.8, label=f'Trend: {z[0]:.2f} mph/inning')
    ax.legend()

plt.tight_layout()
plt.show()

print("\n=== Velocity by Inning (All Pitches) ===")
print(inning_velo.to_string(index=False))

In [ ]:
# Pitch count effect within game
pitch_count_effect = con.execute("""
    WITH pitch_seq AS (
        SELECT
            game_pk,
            pitch_type,
            release_speed,
            ROW_NUMBER() OVER(PARTITION BY game_pk ORDER BY at_bat_number, pitch_number) as pitch_num_in_game
        FROM df
        WHERE pitch_type = 'FF' OR pitch_type = (SELECT pitch_type FROM df WHERE pitch_type IS NOT NULL GROUP BY pitch_type ORDER BY COUNT(*) DESC LIMIT 1)
    )
    SELECT
        CASE
            WHEN pitch_num_in_game <= 25 THEN '1-25'
            WHEN pitch_num_in_game <= 50 THEN '26-50'
            WHEN pitch_num_in_game <= 75 THEN '51-75'
            WHEN pitch_num_in_game <= 100 THEN '76-100'
            ELSE '100+'
        END as pitch_range,
        COUNT(*) as pitches,
        ROUND(AVG(release_speed), 1) as avg_velo
    FROM pitch_seq
    GROUP BY pitch_range
    ORDER BY pitch_range
""").df()

print("\n=== Velocity by Pitch Count in Game ===")
print(pitch_count_effect.to_string(index=False))

## 6. Count-Based Pitch Selection

In [ ]:
# Pitch selection by count situation
count_analysis = con.execute("""
    SELECT
        CASE
            WHEN balls > strikes THEN 'Behind'
            WHEN strikes > balls THEN 'Ahead'
            WHEN balls = 3 AND strikes = 2 THEN 'Full Count'
            ELSE 'Even'
        END as count_situation,
        pitch_type,
        COUNT(*) as pitches,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(PARTITION BY
            CASE
                WHEN balls > strikes THEN 'Behind'
                WHEN strikes > balls THEN 'Ahead'
                WHEN balls = 3 AND strikes = 2 THEN 'Full Count'
                ELSE 'Even'
            END
        ), 1) as pct
    FROM df
    WHERE pitch_type IS NOT NULL
    GROUP BY count_situation, pitch_type
    ORDER BY count_situation, pitches DESC
""").df()

print("=== Pitch Selection by Count Situation ===")
print(count_analysis.to_string(index=False))

In [ ]:
# Two-strike pitch selection (putaway pitches)
two_strike = con.execute("""
    SELECT
        pitch_type,
        COUNT(*) as pitches,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 1) as pct,
        SUM(CASE WHEN description IN ('swinging_strike', 'swinging_strike_blocked', 'called_strike') THEN 1 ELSE 0 END) as strikes,
        ROUND(100.0 * SUM(CASE WHEN description IN ('swinging_strike', 'swinging_strike_blocked') THEN 1 ELSE 0 END) / COUNT(*), 1) as whiff_pct
    FROM df
    WHERE strikes = 2 AND pitch_type IS NOT NULL
    GROUP BY pitch_type
    ORDER BY pitches DESC
""").df()

print("\n=== Two-Strike Pitch Selection (Putaway) ===")
print(two_strike.to_string(index=False))

## 7. Whiff Rate Analysis

In [ ]:
# Whiff rate by pitch type
whiff_analysis = con.execute("""
    SELECT
        pitch_type,
        COUNT(*) as total_pitches,
        SUM(CASE WHEN description LIKE '%swing%' THEN 1 ELSE 0 END) as swings,
        SUM(CASE WHEN description IN ('swinging_strike', 'swinging_strike_blocked') THEN 1 ELSE 0 END) as whiffs,
        ROUND(100.0 * SUM(CASE WHEN description IN ('swinging_strike', 'swinging_strike_blocked') THEN 1 ELSE 0 END) /
              NULLIF(SUM(CASE WHEN description LIKE '%swing%' OR description IN ('foul', 'foul_tip', 'foul_bunt') THEN 1 ELSE 0 END), 0), 1) as whiff_rate,
        ROUND(AVG(release_speed), 1) as avg_velo
    FROM df
    WHERE pitch_type IS NOT NULL
    GROUP BY pitch_type
    ORDER BY whiff_rate DESC
""").df()

print("=== Whiff Rate by Pitch Type ===")
print(whiff_analysis.to_string(index=False))

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.RdYlGn(whiff_analysis['whiff_rate'] / whiff_analysis['whiff_rate'].max())
bars = ax.barh(whiff_analysis['pitch_type'], whiff_analysis['whiff_rate'], color=colors)
ax.set_xlabel('Whiff Rate (%)')
ax.set_title('Yu Darvish 2025 - Whiff Rate by Pitch Type')
ax.bar_label(bars, fmt='%.1f%%')
plt.tight_layout()
plt.show()

## 8. Batted Ball / Damage Analysis

In [ ]:
# Batted ball analysis by pitch type
batted_ball = con.execute("""
    SELECT
        pitch_type,
        COUNT(*) as batted_balls,
        ROUND(AVG(launch_speed), 1) as avg_exit_velo,
        ROUND(AVG(launch_angle), 1) as avg_launch_angle,
        SUM(CASE WHEN launch_speed >= 95 THEN 1 ELSE 0 END) as hard_hit,
        ROUND(100.0 * SUM(CASE WHEN launch_speed >= 95 THEN 1 ELSE 0 END) / COUNT(*), 1) as hard_hit_pct,
        SUM(CASE WHEN events = 'home_run' THEN 1 ELSE 0 END) as hr_allowed
    FROM df
    WHERE launch_speed IS NOT NULL AND pitch_type IS NOT NULL
    GROUP BY pitch_type
    ORDER BY avg_exit_velo DESC
""").df()

print("=== Batted Ball Analysis by Pitch Type ===")
print(batted_ball.to_string(index=False))

In [ ]:
# Exit velocity distribution
exit_velo_data = df[df['launch_speed'].notna()]['launch_speed']

fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(exit_velo_data, bins=30, edgecolor='black', alpha=0.7)
ax.axvline(95, color='red', linestyle='--', label='Hard Hit (95+ mph)')
ax.axvline(exit_velo_data.mean(), color='blue', linestyle='--', label=f'Mean: {exit_velo_data.mean():.1f} mph')
ax.set_xlabel('Exit Velocity (mph)')
ax.set_ylabel('Count')
ax.set_title('Yu Darvish 2025 - Exit Velocity Distribution')
ax.legend()
plt.tight_layout()
plt.show()

## 9. L/R Splits

In [ ]:
# Performance vs Left and Right handed batters
lr_splits = con.execute("""
    SELECT
        stand as batter_side,
        COUNT(*) as pitches,
        COUNT(DISTINCT CASE WHEN events IS NOT NULL THEN game_pk || '-' || at_bat_number END) as PA,
        SUM(CASE WHEN events IN ('single', 'double', 'triple', 'home_run') THEN 1 ELSE 0 END) as hits,
        SUM(CASE WHEN events = 'strikeout' THEN 1 ELSE 0 END) as strikeouts,
        ROUND(AVG(release_speed), 1) as avg_velo,
        ROUND(100.0 * SUM(CASE WHEN description IN ('swinging_strike', 'swinging_strike_blocked') THEN 1 ELSE 0 END) /
              NULLIF(SUM(CASE WHEN description LIKE '%swing%' OR description IN ('foul', 'foul_tip') THEN 1 ELSE 0 END), 0), 1) as whiff_rate
    FROM df
    WHERE stand IS NOT NULL
    GROUP BY stand
""").df()

print("=== L/R Splits ===")
print(lr_splits.to_string(index=False))

In [ ]:
# Pitch mix vs L/R
lr_pitch_mix = con.execute("""
    SELECT
        stand as batter_side,
        pitch_type,
        COUNT(*) as pitches,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(PARTITION BY stand), 1) as pct
    FROM df
    WHERE stand IS NOT NULL AND pitch_type IS NOT NULL
    GROUP BY stand, pitch_type
    ORDER BY stand, pitches DESC
""").df()

print("\n=== Pitch Mix vs L/R ===")
print(lr_pitch_mix.to_string(index=False))

# Side by side bar chart
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for i, side in enumerate(['L', 'R']):
    data = lr_pitch_mix[lr_pitch_mix['batter_side'] == side]
    axes[i].barh(data['pitch_type'], data['pct'], color='steelblue' if side == 'L' else 'coral')
    axes[i].set_xlabel('Usage %')
    axes[i].set_title(f'vs {"Left" if side == "L" else "Right"} Handed Batters')
    axes[i].set_xlim(0, max(lr_pitch_mix['pct']) * 1.1)

plt.suptitle('Yu Darvish 2025 - Pitch Mix by Batter Handedness')
plt.tight_layout()
plt.show()

## 10. Strike Zone Heatmap

In [ ]:
# Strike zone location heatmap
from pybaseball.plotting import plot_strike_zone

# Filter valid data
df_zone = df.dropna(subset=['plate_x', 'plate_z', 'pitch_type'])

if len(df_zone) > 0:
    plot_strike_zone(df_zone, title='Yu Darvish 2025 - Pitch Locations', colorby='pitch_type')
    plt.show()
else:
    print("No valid strike zone data available.")

In [ ]:
# Heatmap of pitch locations
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for i, side in enumerate(['L', 'R']):
    data = df[(df['stand'] == side) & df['plate_x'].notna() & df['plate_z'].notna()]
    if len(data) > 0:
        axes[i].hexbin(data['plate_x'], data['plate_z'], gridsize=20, cmap='YlOrRd', mincnt=1)
        # Strike zone box
        rect = plt.Rectangle((-0.85, 1.5), 1.7, 2.0, fill=False, edgecolor='black', linewidth=2)
        axes[i].add_patch(rect)
        axes[i].set_xlim(-2, 2)
        axes[i].set_ylim(0, 5)
        axes[i].set_xlabel('Plate X')
        axes[i].set_ylabel('Plate Z')
        axes[i].set_title(f'vs {"LHB" if side == "L" else "RHB"}')
        axes[i].set_aspect('equal')

plt.suptitle('Yu Darvish 2025 - Pitch Location Heatmap')
plt.tight_layout()
plt.show()

## 11. Summary & Insights

In [ ]:
# Generate summary insights
print("="*60)
print("YU DARVISH 2025 SEASON ANALYSIS SUMMARY")
print("="*60)

# Best pitch by whiff rate
best_whiff = whiff_analysis.iloc[0] if len(whiff_analysis) > 0 else None
if best_whiff is not None:
    print(f"\n[Best Pitch by Whiff Rate]")
    print(f"  {best_whiff['pitch_type']}: {best_whiff['whiff_rate']}% whiff rate")

# Velocity trend
if len(ff_data) > 1:
    velo_change = ff_data['avg_velo'].iloc[-1] - ff_data['avg_velo'].iloc[0]
    print(f"\n[Fastball Velocity Trend by Inning]")
    print(f"  1st inning: {ff_data['avg_velo'].iloc[0]} mph")
    print(f"  Last inning: {ff_data['avg_velo'].iloc[-1]} mph")
    print(f"  Change: {velo_change:+.1f} mph")

# L/R split
print(f"\n[L/R Split Summary]")
for _, row in lr_splits.iterrows():
    print(f"  vs {row['batter_side']}HB: {row['PA']} PA, {row['strikeouts']} K, Whiff {row['whiff_rate']}%")

print("\n" + "="*60)